new calculator

In [ ]:
from decimal import Decimal, getcontext, ROUND_HALF_UP
# High precision for internal math
getcontext().prec = 28

def mip_24_months(
    cx_ltv: int,
    monthly_roi,
    secure_ltv: int,
    pf_min=None,
    pf_max=None,
    fc=None,
    ts_min: int = None,
    ts_max: int = None,
):
    """MIP 24-month calculator using Decimal (exact math)"""

    # ---- Convert inputs to Decimal ----
    cx_ltv = Decimal(cx_ltv)
    secure_ltv = Decimal(secure_ltv)
    monthly_roi = Decimal(str(monthly_roi))

    pf_min = Decimal(str(pf_min)) if pf_min is not None else None
    pf_max = Decimal(str(pf_max)) if pf_max is not None else None
    fc = Decimal(str(fc)) if fc is not None else None

    # ---- Constants ----
    tenure = Decimal("24")
    secure_irr = Decimal("14.0")
    secure_pf = Decimal("0.00")
    TWOPLACES = Decimal("0.01")

    # ---- LTV calculation ----
    unsecure_ltv = cx_ltv - secure_ltv

    # ---- Interest calculation ----
    annual_irr = monthly_roi * Decimal("12")

    # ---- Unsecured IRR ----
    unsec_irr = (
        (cx_ltv * annual_irr) - (secure_ltv * secure_irr)
    ) / unsecure_ltv

    # ---- Flexi PF calculation ----
    min_percent_unsecure = Decimal("0.00")
    max_percent_unsecure = Decimal("0.00")

    if pf_min is not None:
        min_percent_unsecure = (
            (pf_min * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    if pf_max is not None:
        max_percent_unsecure = (
            (pf_max * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    # ---- FC calculation ----
    fc_charges = Decimal("0.00")
    if fc is not None and unsecure_ltv != 0:
        fc_charges = (
            (fc * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    # ---- FINAL ROUNDING and output----
    return (
        annual_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        unsecure_ltv,
        unsec_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        min_percent_unsecure.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        max_percent_unsecure.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        fc_charges.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        secure_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
    )


In [12]:
x=list(mip_24_months(cx_ltv=84,                  #Trial run of the  MIP calculator
              monthly_roi=1.59, 
            secure_ltv=71,      
            pf_min=0.7,
            ts_min=30000,
            ts_max=299999))
print("the annual_irr is ",x[0],type(x[0]))
print("the unsecure_ltv is ",x[1],type(x[1]))
print("the unsecure_irr is ",x[2],type(x[2]))
print("the min_percent_unsecure is ",x[3],type(x[3]))
print("the max_percent_unsecure is ",x[4],type(x[4]))
print("the fc_charges is ",x[5],type(x[5]))
print("the secure_irr is ",x[6],type(x[6]))


the annual_irr is  19.08 <class 'decimal.Decimal'>
the unsecure_ltv is  13 <class 'decimal.Decimal'>
the unsecure_irr is  46.82 <class 'decimal.Decimal'>
the min_percent_unsecure is  4.52 <class 'decimal.Decimal'>
the max_percent_unsecure is  0.00 <class 'decimal.Decimal'>
the fc_charges is  0.00 <class 'decimal.Decimal'>
the secure_irr is  14.00 <class 'decimal.Decimal'>


In [ ]:
def scheme_validator(
    ts_min: int,
    ts_max: int,
    cx_ltv: int,
    secure_ltv: int,
    monthly_roi,
    annual_irr,
    unsec_irr,
    pf_min=None,
    pf_max=None,
    fc=None,
):
    errors = []

    # ---- Convert to Decimal ----
    cx_ltv = Decimal(cx_ltv)
    secure_ltv = Decimal(secure_ltv)
    monthly_roi = Decimal(str(monthly_roi))
    annual_irr = Decimal(str(annual_irr))
    unsec_irr = Decimal(str(unsec_irr))

    pf_min = Decimal(str(pf_min)) if pf_min is not None else None
    pf_max = Decimal(str(pf_max)) if pf_max is not None else None
    fc = Decimal(str(fc)) if fc is not None else None

    # Constants
    secure_irr = Decimal("14.0")
    secure_pf = Decimal("0.00")

    #ticket size
    if ts_min is None or ts_max is None or ts_min >= ts_max:
        errors.append("Ticket Size range is invalid")

    # LTV checks
    
    if cx_ltv <= 0:
        errors.append("CX LTV must be greater than 0")

    if secure_ltv < 0 or secure_ltv > cx_ltv:
        errors.append("Secure LTV must be between 0 and CX LTV")

    unsecure_ltv = cx_ltv - secure_ltv
    if unsecure_ltv <= 0:
        errors.append("Unsecure LTV must be greater than 0")

    # ------------------------
    # Reverse check 1: annual ↔ monthly
    # ------------------------
    if annual_irr / Decimal("12") != monthly_roi:
        errors.append(
            f"Monthly ROI reverse check failed "
            f"(expected {annual_irr / Decimal('12')}, got {monthly_roi})"
        )

    # ------------------------
    # Reverse check 2: unsecure IRR ↔ annual
    # ------------------------
    derived_annual = (
        (unsec_irr * unsecure_ltv) + (secure_ltv * secure_irr)
    ) / cx_ltv

    if derived_annual != annual_irr:
        errors.append(
            f"Unsecure IRR reverse check failed "
            f"(expected {derived_annual}, got {annual_irr})"
        )

    # ------------------------
    # Reverse check 3: PF MIN
    # ------------------------
    if pf_min is not None:
        derived_pf_min = (
            (pf_min * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

        reconstructed_pf_min = (
            (derived_pf_min * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv

        if reconstructed_pf_min != pf_min:
            errors.append("PF Min reverse check failed")

    # ------------------------
    # Reverse check 4: PF MAX
    # ------------------------
    if pf_max is not None:
        derived_pf_max = (
            (pf_max * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

        reconstructed_pf_max = (
            (derived_pf_max * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv

        if reconstructed_pf_max != pf_max:
            errors.append("PF Max reverse check failed")

    # ------------------------
    # Reverse check 5: FC
    # ------------------------
    if fc is not None:
        derived_fc = (
            (fc * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

        reconstructed_fc = (
            (derived_fc * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv

        if reconstructed_fc != fc:
            errors.append("FC reverse check failed")

    # ------------------------
    # Final
    # ------------------------
    if errors:
        return {
            "status": "INVALID",
            "errors": errors
        }

    return {
        "status": "VALID"
    }
